In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb

# test params from https://www.kaggle.com/hhstrand/oof-stacking-regime/code

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
    if '3_feat' in feat or 'tfidf' in feat:
        continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/fasttext_cnn2d_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v2_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cudnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_gru_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_lstm_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn2d_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v2_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cudnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_gru_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_lstm_v1_4_feat.pkl
(159571, 6) (15

In [5]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

test_pred = np.zeros((153164,6))
for i in range(6):
    m = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, 
                       boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, 
                       colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
    
    m.fit(train_x,train_y[:,i])
    test_pred[:,i] = m.predict_proba(test_x)[:,1]
    train_pred = m.predict_proba(train_x)
    print('col',i)
    print(log_loss(train_y[:,i],train_pred),roc_auc_score(train_y[:,i],train_pred[:,1]))
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/lgb2_log_csv_fold10_stratified.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


col 0
0.0721149353323 0.989887713353


/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


col 1
0.0174716538417 0.994619527144


/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


...

col 3
0.00461924414571 0.9988730704


/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


col 4
0.0502938595791 0.991618655685


/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


col 5
0.0145489629187 0.995768142691
                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.998782      0.332471  0.980539  0.172936  0.925005   
1  0000247867823ef7  0.000407      0.000012  0.000045  0.000007  0.000025   
2  00013b17ad220c46  0.000379      0.000011  0.000079  0.000011  0.000136   
3  00017563c3f7919a  0.000136      0.000012  0.000037  0.000011  0.000027   
4  00017695ad8997eb  0.001845      0.000020  0.000212  0.000012  0.000252   

   identity_hate  
0       0.440222  
1       0.000019  
2       0.000051  
3       0.000026  
4       0.000077  
save done
